# Activity 2: Whole-Genome Sequencing (WGS) Analysis

In this activity, we will analyze whole-genome sequencing data from the [SK-BR-3 breast cancer cell line](https://www.cellosaurus.org/CVCL_0033). We have datasets from Illumina and Oxford Nanopore Technologies (ont) sequencing platforms.

We will perform the following steps:

1. Compare read statistics between sequencing platforms
2. Evaluate alignment quality
3. Call structural variants


## Before you start

Make sure...

1. You are in jupyter notebook mode, not jupyter lab 
2. You have the bootcamp kernel active. If not, do `Kernel` > `Change Kernel` > `bootcamp` 
3. You run the cells below to navigate to the correct directory and prevent output wrapping so we can easily visualize the outputs of our commands.

In [ ]:
%% bash

cd ~/bootcamp-02-sequencing/02-whole-genome

In [ ]:
%%html

<style>
div.output_area pre {
    white-space: pre;
}
</style>


## 1. Compare read statistics between sequencing platforms

You can find FASTQ files containing our WGS reads in the directory shown below

```bash
data/
├── wgs_illumina_R1.fastq.gz
├── wgs_illumina_R2.fastq.gz
├── wgs_ont.fastq.gz
└── wgs_pacbio.fastq.gz
```

Let's compare the reads between different platforms.  We have already run `fastqc` and saved the reports. You can find the resultant HTML files at `~/bootcamp-02-sequencing/02-whole-genome/data/wgs*fastqc.html` and open them as you did in the first activity.

To generate a single report for all the FASTQ files, we can use the amazing [multiqc](https://multiqc.info/).


In [ ]:
%%bash

multiqc data


Take a look at the reports and consider the following questions

1. Which platform has the longest reads? 
2. Which platform has the highest quality reads? 
3. What advantages/disadvantages do you think these features might have for variant calling?

While these three platforms have been the most commmonly used in the last 5-10 years (with Illumina occupying 90% of the market), sequencing technology is currently experiencing exciting growth! You can find a comprehensive and up-to-date comparison of current platforms on [Albert Vilella](https://twitter.com/albertvilella?s=21&t=qY5fTbtw_DsgCnpOqBQPdg)'s great [Next-Generation-Sequencing Google Sheet](https://docs.google.com/spreadsheets/d/1GMMfhyLK0-q8XkIo3YxlWaZA5vVMuhU1kg41g4xLkXc/edit?hl=en_GB&hl=en_GB#gid=1569422585).

## 2. Evaluate alignment quality

We have already aligned the reads to the genome, which are stored in the BAM files shown below.

```bash
data/
├── wgs_illumina.bam
└── wgs_ont.bam
```

Let's take a look at some alignment statistics using [`samtools`](http://www.htslib.org/doc/samtools.html). Running `flagstat` will take a couple minutes on each of these files.

Considering the following questions

1. Which types of flags are present at higher/lower rates between platforms? Why do you think this is?

In [ ]:
%%time
%%bash

# use 8 threads to speed up computation
samtools flagstat -@ 8 data/wgs_illumina.bam 

In [ ]:
%%time
%%bash

# use 8 threads to speed up computation
samtools flagstat -@ 8 data/wgs_ont.bam 

### WGS aligment visualization

Like we did with our whole-exome sequencing data, let's visualize the alignment along a known tumor suppressor *TP53* (aka p53) using the interactive genomics viewer (IGV). Run the code cell below. After the viewer loads, zoom in to view the reads and coverage. (We won't load the PacBio BAM since the viewer will time-out)

In [ ]:
# Must be in Jupyter notebook (not lab) for this to work
import igv_notebook
import os
igv_notebook.init() # initialize igv

browser = igv_notebook.Browser(
    {
        "genome": "hg19",
        "locus": "17:7,569,893-7,579,467" # TP53 gene
    }
)

browser.load_track(
    {
        "name": "Illumina",
        "path": "data/wgs_illumina.bam",
        "indexPath": "data/wgs_illumina.bam.bai",
        "format": "bam",
        "type": "alignment"
    })


browser.load_track(
    {
        "name": "ONT",
        "path": "data/wgs_ont.bam",
        "indexPath": "data/wgs_ont.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

## 3. Call structural variants (SVs) from long- and short-read alignments

One major advantage of long-read sequencing is the ability to call structural variants (SVs) with higher accuracy. We have run  the long-read structural variant caller [Sniffles2](https://github.com/fritzsedlazeck/Sniffles) on the nanpore BAM file to call SVs. To compare to SVs calls from short-read data, we used the short-read structural variant caller [Manta](https://github.com/Illumina/manta) on the Illumina BAM file. The outputs of these runs are stored in the VCF files shown below.

```bash
data/
├── wgs_illumina_manta.vcf.gz
└── wgs_ont_sniffles.vcf.gz
```

Let's first compare the SVs called on the short and long reads. From what you've learned so far, write your own code below to compare the VCFs. *Hint*: use [`bcftools`](http://samtools.github.io/bcftools/bcftools.html) 

Consider the following questions:

1. How are SVs annotated in each VCF file?
2. How many SVs were discovered from each alignment?
3. Where are largest SVs? Where are the smallest

In [ ]:
# write your code here